# Spark session

In [2]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

# spark = sparknlp.start()

spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","6G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.1.0")\
    .getOrCreate()

parqDF = spark.read.parquet("./tmp/news.parquet")

parqDF.show()

22/10/09 18:45:39 WARN Utils: Your hostname, zainab-ThinkPad-T560 resolves to a loopback address: 127.0.1.1; using 192.168.178.29 instead (on interface wlp4s0)
22/10/09 18:45:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/zainab/miniconda3/envs/sparknlp/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/zainab/.ivy2/cache
The jars for the packages stored in: /home/zainab/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-73c4868d-1f11-48d8-b034-8062a723edf0;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.1.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlom

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _id|             content|                hash|                link|           timestamp|               title|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|{6330c4d7e5ec028a...|ISLAMABAD: The nu...|d7d93961dc04faa43...|https://dailytime...|2022-09-25T00:00:...|104 dengue cases ...|
|{6330c4d7e5ec028a...|One killed, two w...|f4b3fb3dca7ab9b57...|https://dailytime...|2022-09-25T00:00:...|One killed, two w...|
|{6330c4d7e5ec028a...|Pakistan Navy’s f...|30a4964cf67223cf0...|https://dailytime...|2022-09-25T00:00:...|Armed forces cont...|
|{6330c4d7e5ec028a...|Karachi: The Fede...|5c1f6085633516241...|https://dailytime...|2022-09-25T00:00:...|Director State Of...|
|{6330c4d7e5ec028a...|Former prime mini...|d969decd78ce4c766...|https://dailytime...|2022-09-25T00:00:..

In [8]:
# from pyspark.sql.functions import lit
# df.withColumn("bonus_percent", lit(0.3))

loadedPipeline = PipelineModel.load("./trained_pipelines/classifier_pipeline")
predClassifier = loadedPipeline.transform(parqDF)
predClassifier = predClassifier.select("hash","title","content","link","timestamp","prediction")
parqDF = predClassifier.filter( (predClassifier.prediction  == "1.0"))

In [9]:
parqDF.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                hash|               title|             content|                link|           timestamp|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|f4b3fb3dca7ab9b57...|One killed, two w...|One killed, two w...|https://dailytime...|2022-09-25T00:00:...|       1.0|
|76ba7a03b94e9e155...|16 dead, 901 inju...|The Punjab Emerge...|https://dailytime...|2022-09-26T00:00:...|       1.0|
|2e59be1e208ef0243...|Two robbers kille...|Two robbers were ...|https://dailytime...|2022-09-25T00:00:...|       1.0|
|bbf9db6a264380f68...|Street crimes in ...|Lahore: Street cr...|https://dailytime...|2022-09-24T00:00:...|       1.0|
|6c16b862add6e696c...|Man arrested in K...|Karachi: Man arre...|https://dailytime...|2022-09-24T00:00:...|       1.0|
|0ed4aff27c8cd199a...|Director of Syste...|Abdul Qadir, 

In [2]:
MODEL_NAME='sentimentdl_use_twitter'

In [105]:
documentAssembler = DocumentAssembler()\
    .setInputCol("content")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ | ]sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
Download done! Loading the resource.
[OK!]


In [106]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(parqDF)

In [107]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _id|             content|                hash|                link|           timestamp|               title|            document| sentence_embeddings|           sentiment|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|{6330c4d7e5ec028a...|ISLAMABAD: The nu...|d7d93961dc04faa43...|https://dailytime...|2022-09-25T00:00:...|104 dengue cases ...|[{document, 0, 32...|[{sentence_embedd...|[{category, 0, 32...|
|{6330c4d7e5ec028a...|One killed, two w...|f4b3fb3dca7ab9b57...|https://dailytime...|2022-09-25T00:00:...|One killed, two w...|[{document, 0, 32...|[{sentence_embedd...|[{category, 0, 32...|
|{6330c4d7e5ec028a...|Pakistan Navy’s f...|30

In [108]:
result.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- content: string (nullable = true)
 |-- hash: string (nullable = true)
 |-- link: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- title: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = f

In [110]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

tmp = result.select('title', 'sentiment.result')
tmp = tmp.withColumn('result', F.explode(tmp.result))
tmp.show(100, truncate=False)




+-------------------------------------------------------------------------------+--------+
|title                                                                          |result  |
+-------------------------------------------------------------------------------+--------+
|104 dengue cases reported in Islamabad today                                   |negative|
|One killed, two wounded in Karachi incidents                                   |negative|
|Armed forces continue rescue relief operations in flood hit areas              |negative|
|Director State Office suspended for misbehaving with woman                     |negative|
|If I’m assassinated                                                            |negative|
|PTI’s US policy ‘most uncalled for’                                            |negative|
|Pakistan at war from within                                                    |negative|
|Dar set to take over as finance minister                                       |negative|

# Topic Model

In [95]:
parqDF = spark.read.parquet("./tmp/news.parquet")

parqDF.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _id|             content|                hash|                link|           timestamp|               title|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|{6330c4d7e5ec028a...|ISLAMABAD: The nu...|d7d93961dc04faa43...|https://dailytime...|2022-09-25T00:00:...|104 dengue cases ...|
|{6330c4d7e5ec028a...|One killed, two w...|f4b3fb3dca7ab9b57...|https://dailytime...|2022-09-25T00:00:...|One killed, two w...|
|{6330c4d7e5ec028a...|Pakistan Navy’s f...|30a4964cf67223cf0...|https://dailytime...|2022-09-25T00:00:...|Armed forces cont...|
|{6330c4d7e5ec028a...|Karachi: The Fede...|5c1f6085633516241...|https://dailytime...|2022-09-25T00:00:...|Director State Of...|
|{6330c4d7e5ec028a...|Former prime mini...|d969decd78ce4c766...|https://dailytime...|2022-09-25T00:00:..

In [96]:
# Spark NLP requires the input dataframe or column to be converted to document. 
document_assembler = DocumentAssembler() \
    .setInputCol("content") \
    .setOutputCol("document") \
    .setCleanupMode("shrink")
# Split sentence to tokens(array)
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
# clean unwanted characters and garbage
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")
# remove stopwords
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)
# stem the words to bring them to the root form.
stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")
# Finisher is the most important annotator. Spark NLP adds its own structure when we convert each row in the dataframe to document. Finisher helps us to bring back the expected structure viz. array of tokens.
finisher = Finisher() \
    .setInputCols(["stem"]) \
    .setOutputCols(["tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)
# We build a ml pipeline so that each phase can be executed in sequence. This pipeline can also be used to test the model. 
nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher])
# train the pipeline
nlp_model = nlp_pipeline.fit(parqDF)
# apply the pipeline to transform dataframe.
processed_df  = nlp_model.transform(parqDF)
# nlp pipeline create intermediary columns that we dont need. So lets select the columns that we need
tokens_df = processed_df.select('hash','title','tokens')
tokens_df.show(2, truncate=False)

+----------------------------------------------------------------+--------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|hash                                                            |title                                       |tokens                                                                                                                                                                                                                                                 |
+----------------------------------------------------------------+--------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [97]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=500, minDF=3.0)
# train the model
cv_model = cv.fit(tokens_df)
# transform the data. Output column name will be features.
vectorized_tokens = cv_model.transform(tokens_df)
vectorized_tokens.show()

+--------------------+--------------------+--------------------+--------------------+
|                hash|               title|              tokens|            features|
+--------------------+--------------------+--------------------+--------------------+
|d7d93961dc04faa43...|104 dengue cases ...|[islamabad, numbe...|(500,[7,15,20,23,...|
|f4b3fb3dca7ab9b57...|One killed, two w...|[on, kill, two, w...|(500,[22,49,52,59...|
|30a4964cf67223cf0...|Armed forces cont...|[pakistan, navi, ...|(500,[0,3,6,12,18...|
|5c1f6085633516241...|Director State Of...|[karachi, feder, ...|(500,[11,22,26,44...|
|d969decd78ce4c766...| If I’m assassinated|[former, prime, m...|(500,[0,1,2,4,5,6...|
|ca97e188dd158cdc8...|PTI’s US policy ‘...|[shahbaz, sharif,...|(500,[3,27,28,34,...|
|ad86f452be2a31739...|Pakistan at war f...|[war, world, base...|(500,[0,74,77,199...|
|8ededb8caf70e56ac...|Dar set to take o...|[pmln, leader, is...|(500,[2,8,11,27,2...|
|fe66ae9402433c5bc...|President Alvi ag...|[presid, dr

In [102]:
from pyspark.ml.clustering import LDA
num_topics = 100
lda = LDA(k=num_topics, maxIter=10)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -395043.91692297254
The upper bound on perplexity: 9.707431304164457


In [99]:
# extract vocabulary from CountVectorizer
vocab = cv_model.vocabulary
topics = model.describeTopics()   
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
pakistan
flood
minist
prime
nation
countri
govern
said
shehbaz
unit
*************************
topic: 1
*************************
pakistan
court
islamabad
karachi
justic
said
pti
minist
case
leader
*************************
topic: 2
*************************
said
khan
minist
imran
chief
flood
pakistan
chairman
punjab
area
*************************


In [103]:
from pyspark.sql import types as T
def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))
num_top_words = 10

topics = model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)

+-----+-------------------------------------------------------------------------------+
|topic|                                                                     topicWords|
+-----+-------------------------------------------------------------------------------+
|    0|    [afghanistan, unit, dengu, cm, author, last, council, polit, provinci, hec]|
|    1|       [busi, ahm, suprem, saturdai, report, victim, toll, assist, view, first]|
|    2|    [occas, staff, balochistan, pti, close, mani, children, ecp, imran, direct]|
|    3|[pakistan, minist, said, prime, chief, nation, islamabad, court, govern, flood]|
|    4|    [humanitarian, last, tv, queen, role, discuss, guterr, complet, dai, recov]|
|    5|        [observ, complet, high, ne, byelect, lake, major, capit, disast, india]|
|    6|     [oper, terrorist, kill, khan, relief, pakistan, flood, brief, chief, area]|
|    7|                [ga, hear, bail, coa, bank, notic, set, program, reject, heard]|
|    8|  [bilaw, afghanistan, pe

In [105]:
topics.select('topic', 'topicWords').write.save("./topics/news_transformed.parquet", format="parquet")

# News Classify

In [10]:
### Select Model
# model_age_news = "bert_sequence_classifier_age_news"
model_emotion = "bert_sequence_classifier_emotion"

In [11]:
model_dict = {
            #   model_age_news :parqDF,
              model_emotion: parqDF
              }

In [12]:
def run_pipeline(model, df, results):  
  document_assembler = DocumentAssembler() \
      .setInputCol('content') \
      .setOutputCol('document')

  tokenizer = Tokenizer() \
      .setInputCols(['document']) \
      .setOutputCol('token')

  sequenceClassifier = BertForSequenceClassification\
        .pretrained(model, 'en') \
        .setInputCols(['token', 'document']) \
        .setOutputCol('pred_class')

  pipeline = Pipeline(stages=[document_assembler, tokenizer, sequenceClassifier])
  results[model]=(pipeline.fit(df).transform(df))

In [13]:
results = {}
for model, text in zip(model_dict.keys(),model_dict.values()):
  run_pipeline(model, text, results)

bert_sequence_classifier_emotion download started this may take some time.
Approximate size to download 391.1 MB
[ / ]bert_sequence_classifier_emotion download started this may take some time.
Approximate size to download 391.1 MB
Download done! Loading the resource.
[ | ]

2022-10-09 19:09:00.258529: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ / ]

2022-10-09 19:09:03.888772: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% of free system memory.


[OK!]


In [14]:
for model_name, result in zip(results.keys(),results.values()):  
  res = result.select(F.explode(F.arrays_zip(result.document.result, 
                                                  result.pred_class.result,
                                                  result.pred_class.metadata)).alias("col"))\
                  .select(F.expr("col['1']").alias("prediction"),
                          F.expr("col['2']").alias("confidence"),
                          F.expr("col['0']").alias("sentence"))
                  
  if res.count()>1:
    udf_func = F.udf(lambda x,y:  x["Some("+str(y)+")"])
    print("\n",model_name,"\n") 
    res.withColumn('confidence', udf_func(res.confidence, res.prediction)).show(truncate=False)
    print("\n**********************************\n") 


 bert_sequence_classifier_emotion 



+----------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction|confidence|sentence                                                                                                                                                                                                                                                                                                                                                                                                        |
+----------+----------+-------------------------------------------------------------------------------------------------------------------------------

# NER

In [15]:
document = DocumentAssembler()\
    .setInputCol("content")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
  .setInputCols("document")\
  .setOutputCol("token")  

tokenClassifier = BertForTokenClassification.pretrained("bert_token_classifier_ner_btc", "en")\
  .setInputCols("token", "document")\
  .setOutputCol("ner")\
  .setCaseSensitive(True)

ner_converter = NerConverter()\
        .setInputCols(["document","token","ner"])\
        .setOutputCol("ner_chunk")\
        

pipeline =  Pipeline(stages=[document, tokenizer, tokenClassifier, ner_converter])

bert_token_classifier_ner_btc download started this may take some time.
Approximate size to download 385.3 MB
[ | ]bert_token_classifier_ner_btc download started this may take some time.
Approximate size to download 385.3 MB
[ / ]Download done! Loading the resource.
[ — ]

2022-10-09 19:15:21.713993: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 89075712 exceeds 10% of free system memory.


[OK!]


In [16]:
modelNER = pipeline.fit(parqDF)
resultNER = modelNER.transform(parqDF)

In [17]:
resultNER.printSchema()

root
 |-- hash: string (nullable = true)
 |-- title: string (nullable = true)
 |-- content: string (nullable = true)
 |-- link: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- prediction: double (nullable = false)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = 

In [18]:
resultNER.select('title','token.result','ner.result').show(1, truncate=False)


+--------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                       |result                                                                                                                                                                                                                                                                                             

In [19]:
resultNER.select(F.explode(F.arrays_zip(resultNER.ner_chunk.result, resultNER.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label"),
              F.expr("cols['1']['confidence']").alias("confidence")).show(40, truncate=False)

+--------------------------------------+---------+----------+
|chunk                                 |ner_label|confidence|
+--------------------------------------+---------+----------+
|Karachi                               |LOC      |0.9879605 |
|Karachi                               |LOC      |0.98792946|
|Awami Markaz neighborhood             |LOC      |0.7259514 |
|city                                  |LOC      |0.9419481 |
|Punjab Emergency Service              |ORG      |0.59160167|
|PES                                   |ORG      |0.72939134|
|Punjab                                |LOC      |0.94259423|
|Madina Town                           |LOC      |0.9802779 |
|Faisalabad                            |LOC      |0.9846722 |
|Lahore                                |LOC      |0.9703298 |
|Lahore                                |LOC      |0.9735148 |
|city                                  |LOC      |0.5037127 |
|Iqbal Town                            |LOC      |0.9093797 |
|Lahore 

In [23]:
cleanDf = resultNER.select('title',F.explode(F.arrays_zip( resultNER.ner_chunk.result, resultNER.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("title"),
            F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label"),
              F.expr("cols['1']['confidence']").alias("confidence"))

In [25]:
cleanDf.printSchema()

root
 |-- title: string (nullable = true)
 |-- chunk: string (nullable = true)
 |-- ner_label: string (nullable = true)
 |-- confidence: string (nullable = true)



In [26]:
from pyspark.sql.types import FloatType
cleanDf = cleanDf.filter( (cleanDf.ner_label  == "LOC") & (cleanDf.confidence.cast(FloatType())  >= 0.70) )

In [27]:
cleanDfv = cleanDf.groupBy('title').agg(F.collect_set('chunk').alias('chunk'))
cleanDfv.show()

+--------------------+--------------------+
|               title|               chunk|
+--------------------+--------------------+
|Pakistan for defe...|          [Pakistan]|
|22 more die as fl...|          [Pakistan]|
|Senior journalist...|         [Islamabad]|
|Two robbers kille...|        [Faisalabad]|
|Terrorism revisit...|[Kabal tehsil, Ba...|
|Man commits suici...|         [Islamabad]|
|Mentally challeng...|  [Pakistan, Lahore]|
|24 dead after boa...|        [Bangladesh]|
|Street crimes in ...|[Iqbal Town, Lahore]|
|Soldier martyred ...|[Afghanistan, Nor...|
|Two dead in SNGPL...|[Faisalabad, Smal...|
|Employee allegedl...|            [Jhelum]|
|SC issues show ca...|            [Multan]|
|FIR filed against...|[Shahzad Town pol...|
|Man arrested in K...|           [Karachi]|
|Two die in Karach...|[Saudabad, Karach...|
|Court adjourns he...|[Rawalpindi, Ring...|
|Police claims to ...|[Jaranwala, Faisa...|
|Teenage girl shot...|[village 46/KB, F...|
|Three cops injure...|[Kohat dis

In [28]:
cleanDfv.show(20, truncate=False)

+-----------------------------------------------------------------------------+------------------------------------------------------------------+
|title                                                                        |chunk                                                             |
+-----------------------------------------------------------------------------+------------------------------------------------------------------+
|Pakistan for defeating terror by addressing root causes                      |[Pakistan]                                                        |
|22 more die as flood-hit areas still under water                             |[Pakistan]                                                        |
|Senior journalist Ayaz Amir’s daughter-in-law found dead in Islamabad        |[Islamabad]                                                       |
|Two robbers killed, cop injured in encounter                                 |[Faisalabad]                           

In [87]:
cleanDfv.select("title", "chunk").write.save("./tmp2/news_transformed.parquet", format="parquet")

In [74]:
cleanDf.show(50)

+--------------------+--------------------+---------+----------+
|               title|               chunk|ner_label|confidence|
+--------------------+--------------------+---------+----------+
|104 dengue cases ...|           Islamabad|      LOC|  0.970374|
|104 dengue cases ...|           Islamabad|      LOC|0.84157187|
|One killed, two w...|             Karachi|      LOC| 0.9879605|
|One killed, two w...|             Karachi|      LOC|0.98792946|
|One killed, two w...|Awami Markaz neig...|      LOC| 0.7259514|
|One killed, two w...|                city|      LOC| 0.9419481|
|Director State Of...|             Karachi|      LOC| 0.9619475|
| If I’m assassinated|           Islamabad|      LOC| 0.8782349|
| If I’m assassinated|            Pakistan|      LOC|  0.862541|
|Pakistan at war f...|            Pakistan|      LOC| 0.9772868|
|Pakistan at war f...|            Pakistan|      LOC|0.96390104|
|Pakistan at war f...|            Pakistan|      LOC|  0.952971|
|Dar set to take o...|   

In [47]:
resultNER.select(resultNER.ner_chunk.result).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------+
|ner_chunk.result                                                                                                            |
+----------------------------------------------------------------------------------------------------------------------------+
|[ISLAMABAD, Islamabad, federal, Islamabad, federal]                                                                         |
|[Karachi, Karachi, Awami Markaz neighborhood, city]                                                                         |
|[Pakistan Navy’s, National, NFRCC, Navy]                                                                                    |
|[Karachi, Federal government, Federal C, Agha Ayub Jan, Housing, Ayub]                                                      |
|[PTI, Imran Khan, National, Islamabad, PTI, Pakistan]                                                         

In [53]:
# resultNER.select(resultNER.ner.result, array_position(resultNER.ner.result, "B-LOC").alias('loc')).show()
resultNER.select(resultNER.ner_chunk.result,resultNER.ner_chunk.metadata).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ner_chunk.result                                                                                           

In [48]:
cleasnDf = resultNER.select(resultNER.title, resultNER.ner_chunk.result, resultNER.ner_chunk.metadata)

cleasnDf.show()



+--------------------+--------------------+--------------------+
|               title|    ner_chunk.result|  ner_chunk.metadata|
+--------------------+--------------------+--------------------+
|104 dengue cases ...|[ISLAMABAD, Islam...|[{entity -> ORG, ...|
|One killed, two w...|[Karachi, Karachi...|[{entity -> LOC, ...|
|Armed forces cont...|[Pakistan Navy’s,...|[{entity -> ORG, ...|
|Director State Of...|[Karachi, Federal...|[{entity -> LOC, ...|
| If I’m assassinated|[PTI, Imran Khan,...|[{entity -> ORG, ...|
|PTI’s US policy ‘...|    [Shahbaz Sharif]|[{entity -> PER, ...|
|Pakistan at war f...|[world, Pakistan,...|[{entity -> LOC, ...|
|Dar set to take o...|[PML-N, Ishaq Dar...|[{entity -> ORG, ...|
|President Alvi ag...|[Dr, Arif Alvi Sa...|[{entity -> PER, ...|
|Bringing about ch...|[Rana Sanaullah, ...|[{entity -> PER, ...|
|Unprecedented jum...|[Quaid-e-Azam Uni...|[{entity -> ORG, ...|
|Kashmiri proteste...|[Azad, Kashmir, S...|[{entity -> LOC, ...|
|Int’l conference ...|[Al

In [22]:
cleanDf.printSchema()

root
 |-- chunk: string (nullable = true)
 |-- ner_label: string (nullable = true)
 |-- confidence: string (nullable = true)



In [10]:
from pyspark.sql.types import FloatType,IntegerType,BooleanType,DateType
# Convert String to Integer Type
cleanDf = cleanDf.withColumn("confidence",cleanDf.confidence.cast(FloatType()))

In [11]:
cleanDf.printSchema()

root
 |-- chunk: string (nullable = true)
 |-- ner_label: string (nullable = true)
 |-- confidence: float (nullable = true)



In [15]:
from geopy.geocoders import Nominatim
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

def clean_ner(chunk, location, confidence):
    geolocator = Nominatim(user_agent="ehsan")
    if location.casefold() == "LOC".casefold() and confidence > 0.70:
        glocation = geolocator.geocode(chunk)
        if (glocation != None):
            return [glocation.latitude, glocation.longitude]

ner_clean_fn = udf(clean_ner, StringType())

cleanDf = cleanDf.withColumn("cleaned_ner", ner_clean_fn(cleanDf['chunk'], cleanDf['ner_label'], cleanDf['confidence']))
cleanDf.show()

+--------------------+---------+----------+--------------------+
|               chunk|ner_label|confidence|         cleaned_ner|
+--------------------+---------+----------+--------------------+
|           ISLAMABAD|      ORG| 0.5019421|                null|
|           Islamabad|      LOC|  0.970374|[33.6938118, 73.0...|
|             federal|      LOC|0.69779205|                null|
|           Islamabad|      LOC|0.84157187|[33.6938118, 73.0...|
|             federal|      LOC|  0.615035|                null|
|             Karachi|      LOC| 0.9879605|[24.8546842, 67.0...|
|             Karachi|      LOC|0.98792946|[24.8546842, 67.0...|
|Awami Markaz neig...|      LOC| 0.7259514|                null|
|                city|      LOC| 0.9419481|[-33.867926, 151....|
|     Pakistan Navy’s|      ORG|0.68276286|                null|
|            National|      ORG|0.61495835|                null|
|               NFRCC|      ORG|0.68793434|                null|
|                Navy|   